In [1]:
import polars as pl
from pathlib import Path


#Esto para el script de py
#DATA_PATH = Path(__file__).resolve().parents[1] / "data" / "raw" / "emails.csv"
DATA_PATH = Path.cwd().parent / "data" / "raw" / "emails.csv"

print(DATA_PATH)
#Read csv
emails = pl.read_csv(DATA_PATH)
emails.columns

c:\Users\Martín\Desktop\Proyectos Personales\email-ticket-classifier\data\raw\emails.csv


['id_email', 'user_email', 'subject', 'body', 'date', 'language']

In [7]:
CLEAN_SUBJECT_COL = "clean_subject"
CLEAN_BODY_COL = "clean_body"
FULL_TEXT_COL = "full_text"

def clean_expr(expr: pl.Expr) -> pl.Expr:
    return(expr
           .fill_null("")
           .str.to_lowercase()
           .str.strip_chars()
           )

def clean_text(df: pl.DataFrame) -> pl.DataFrame:
    df = df.with_columns(
        clean_expr(pl.col("subject")).alias(CLEAN_SUBJECT_COL),
        clean_expr(pl.col("body")).alias(CLEAN_BODY_COL),
    )

    df = df.with_columns(
        pl.concat_str(
            [pl.col(CLEAN_SUBJECT_COL), pl.col(CLEAN_BODY_COL)],
            separator=" "
        ).alias("full_text")
    )
    
    return df

In [9]:
clean_emails = clean_text(emails)

In [15]:
clean_emails.columns

['id_email',
 'user_email',
 'subject',
 'body',
 'date',
 'language',
 'clean_subject',
 'clean_body',
 'full_text']

In [12]:
import json
from pathlib import Path
DATA_PATH = Path.cwd().parent / "data" / "dictionaries" / "error_patterns_en.json"
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    dict = json.load(f)

In [13]:
dict

{'version': '1.0',
 'last_updated': '2025-01-01',
 'patterns': [{'id': 'ERR_001',
   'pattern': 'error 500',
   'label': 'bug',
   'confidence': 0.95,
   'match_type': 'contains',
   'enabled': True,
   'source': 'manual'},
  {'id': 'ERR_002',
   'pattern': 'error 200',
   'label': 'bug',
   'confidence': 0.95,
   'match_type': 'contains',
   'enabled': False,
   'source': 'manual'},
  {'id': 'ERR_003',
   'pattern': 'crash',
   'label': 'bug',
   'confidence': 0.9,
   'match_type': 'contains',
   'enabled': True,
   'source': 'manual'},
  {'id': 'REQ_001',
   'pattern': 'feature request',
   'label': 'feature_request',
   'confidence': 0.85,
   'match_type': 'contains',
   'enabled': True,
   'source': 'manual'},
  {'id': 'REQ_002',
   'pattern': 'i would like to',
   'label': 'feature_request',
   'confidence': 0.8,
   'match_type': 'contains',
   'enabled': True,
   'source': 'manual'}]}

In [14]:
patterns = dict.get("patterns", [])
enabled = [r for r in patterns if r.get("enabled", False)]

In [15]:
enabled

[{'id': 'ERR_001',
  'pattern': 'error 500',
  'label': 'bug',
  'confidence': 0.95,
  'match_type': 'contains',
  'enabled': True,
  'source': 'manual'},
 {'id': 'ERR_003',
  'pattern': 'crash',
  'label': 'bug',
  'confidence': 0.9,
  'match_type': 'contains',
  'enabled': True,
  'source': 'manual'},
 {'id': 'REQ_001',
  'pattern': 'feature request',
  'label': 'feature_request',
  'confidence': 0.85,
  'match_type': 'contains',
  'enabled': True,
  'source': 'manual'},
 {'id': 'REQ_002',
  'pattern': 'i would like to',
  'label': 'feature_request',
  'confidence': 0.8,
  'match_type': 'contains',
  'enabled': True,
  'source': 'manual'}]